<a href="https://colab.research.google.com/github/Bindulekh/private/blob/main/prompt_engineering_practice_notebook_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prompt Engineering Best Practices for Instruction-Tuned LLM

Have you ever wondered why your interaction with a language model falls short of expectations? The answer may lie in the clarity of your instructions.

Picture this scenario: requesting someone, perhaps a bright but task-unaware individual, to write about a popular figure. It’s not just about the subject; clarity extends to specifying the focus — scientific work, personal life, historical role — and even the desired tone, be it professional or casual. Much like guiding a fresh graduate through the task, offering specific snippets for preparation sets the stage for success.

In this notebook, we’re going to help you make your talks with the language model better by getting really good at giving clear and specific instructions to get the expected output.

1. Setting Up Work Environment

In [ ]:
pip install google-generativeai

In [ ]:
import os

import google.generativeai as genai

In [ ]:
api_key = 'Google API Key'
genai.configure(api_key=api_key)

In [ ]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)
prompt = f"""
Summarize the text into a single sentence."""

contents = ['''
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
''', prompt]

response = model.generate_content(contents)
print(response.text)

You should provide clear and specific instructions to guide the model towards the desired output, using longer prompts when necessary for clarity and context. 



2.2. Ask for a structured output
The next tactic is to ask for a structured output. To make parsing the model outputs easier, it can be helpful to ask for a structured output like HTML or JSON.

So in the prompt, we’re saying generate a list of three made-up book titles along with their authors and genres. Provide them in JSON format with the following keys, book ID, title, author, and genre.

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash',
                              generation_config={"response_mime_type": "application/json"})

prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""

response = model.generate_content(prompt)
print(response.text)


{"books": [{"book_id": "1", "title": "Whispers of the Ancient Forest", "author": "Elara Blackwood", "genre": "Fantasy"}, {"book_id": "2", "title": "The Clockwork Detective", "author": "Jasper Thorne", "genre": "Steampunk"}, {"book_id": "3", "title": "The Last Stargazer", "author": "Anya Petrova", "genre": "Science Fiction"}]}



2.3. Ask the model to check whether conditions are satisfied
The next tactic is to ask the model to check whether conditions are satisfied. If the task makes assumptions that aren’t necessarily satisfied, then we can tell the model to check these assumptions first. Then if they’re not satisfied, indicate this and kind of stop short of a full task completion attempt. You might also consider potential edge cases and how the model should handle them to avoid unexpected errors or results.

Let's take a paragraph describing the steps to make a cup of tea. And then I will copy over the prompt. So the prompt you’ll be provided with text delimited by triple quotes. If it contains a sequence of instructions, rewrite those instructions in the following format and then just the steps written out. If the text does not contain a sequence of instructions, then simply write, no steps provided.

In [ ]:
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config={"response_mime_type" : "text/plain"}
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = model.generate_content(prompt)
print("Completion for Text 1:")
print(response.text)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the boiling water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste (optional). 



Let’s try this same prompt with a different paragraph. This paragraph is just describing a sunny day, it doesn’t have any instructions in it. So, if we take the same prompt we used earlier and instead run it on this text, the model will try and extract the instructions. If it doesn’t find any, we’re going to ask it to just say, no steps provided.



In [ ]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = model.generate_content(prompt)
print("Completion for Text 2:")
print(response.text)


Completion for Text 2:
No steps provided. 



2.4. Few-shot prompting
The final tactic for this principle is what we call few-shot prompting. This is just providing examples of successful executions of the task you want to be performed before asking the model to do the actual task you want it to do.

In this prompt, we’re telling the model that its task is to answer in a consistent style. We have this example of a kind of conversation between a child and a grandparent.

The kind of child who says, teach me about patience. The grandparent responds with these kinds of metaphors. And so, since we’ve kind of told the model to answer in a consistent tone, now we’ve said, teach me about resilience.

Since the model kind of has this few-shot example, it will respond in a similar tone to this next instruction. So, resilience is like a tree that bends with the wind but never breaks, and so on.

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = model.generate_content(prompt)
print(response.text)


<grandparent>: The willow bends low in the wind, but its roots hold firm. The seed buried deep in the earth endures the winter's chill to blossom in spring. The eagle soars above the storm, its wings unyielding in the face of adversity. 



3. Give the model time to think
The second principle is to give the model time to think. Suppose a model is making reasoning errors by rushing to an incorrect conclusion. In that case, you should try reframing the query to request a chain or series of relevant reasoning before the model provides its final answer.

Another way to think about this is that if you give a model a task that’s too complex for it to do in a short amount of time or a small number of words, it may make up a guess that is likely to be incorrect and this would happen to a person too.

If you ask someone to complete a complex math question without time to work out the answer first, they would also likely make a mistake. So, in these situations, you can instruct the model to think longer about a problem, which means it’s spending more computational effort on the task. Let’s go over some tactics for the second principle.

3.1 Specify the steps required to complete a task
The first tactic is to specify the steps required to complete a task. Let's take for example the given prompt which is a description of the story of Jack and Jill. In this prompt, the instructions are to perform the following actions. First, summarize the following text delimited by triple backticks with one sentence. Second, translate the summary into French. Third, list each name in the French summary. Fourth, output a JSON object that contains the following keys, French summary, and num names. And then we want it to separate the answers with line breaks. And so, we add the text, which is just this paragraph.

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.
Text:
```{text}```
"""
response = model.generate_content(prompt_1)
print("Completion for prompt 1:")
print(response.text)

Completion for prompt 1:
Jack and Jill, siblings from a charming village, had a mishap while fetching water but remained undeterred in their adventurous spirit.

Jack et Jill, frère et sœur d'un charmant village, ont eu un accident en allant chercher de l'eau mais sont restés sans peur dans leur esprit aventureux.

- Jack
- Jill

```json
{
"french_summary": "Jack et Jill, frère et sœur d'un charmant village, ont eu un accident en allant chercher de l'eau mais sont restés sans peur dans leur esprit aventureux.",
"num_names": 2
}
```


If we run this you can see that we have the summarized text. Then we have the French translation. Finally, we have the names. It gave the names a title in French. Then we have the JSON that we requested. Let's take another prompt to complete the same task. In this prompt, we will use a format that specifies the output structure for the model because as you notice in this example, this name’s title is in French which we might not necessarily want.

In this prompt, we’re asking for something similar. The beginning of the prompt is the same, we’re just asking for the same steps and then we’re asking the model to use the following format. So we have specified the exact format of text, summary, translation, names, and output JSON. Finally, we start by just saying the text to summarize or we can even just say the text.

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>
Text: <{text}>
"""
response = model.generate_content(prompt_2)
print("\nCompletion for prompt 2:")
print(response.text)



Completion for prompt 2:
Text: <
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled down the hill, with Jill following suit. Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they continued exploring with delight.
>

Summary: Jack and Jill, siblings from a charming village, embark on a water-fetching adventure that ends in a comical tumble down a hill, but their spirits remain unbroken. 

Translation: Jack et Jill, frère et sœur d'un village charmant, partent à l'aventure pour aller chercher de l'eau à un puits situé au sommet d'une colline, mais leur périple se termine par une chute comique en bas de la colline, sans pour autant entamer leur enthousiasme.

Names: Jack, Jill 

Output JSON: 
```json
{
  "french_summary": "Jack et Jill, frère et sœur d'un vill

You can see, that this is the completion and the model has used the format that we asked for. So, we already gave it the text, and then it gave us the summary, the translation, the names, and the output JSON. This is sometimes nice because it’s going to be easier to pass this with code because it kind of has a more standardized format that you can kind of predict.

Also notice that in this case, we’ve used angled brackets as the delimiter instead of triple backticks. You can choose any delimiters that make sense to you, and that make sense to the model.

3.2. Instruct the model to work out its solution before rushing to a conclusion
The next tactic is to instruct the model to work out its own solution before rushing to a conclusion. Sometimes we get better results when we explicitly instruct the models to reason out its own solution before concluding.

This is the same idea that we were discussing before which is giving the model time to work things out before just kind of saying if an answer is correct or not, in the same way that a person would.

In this prompt, we’re asking the model to determine if the student’s solution is correct or not. We have this math question first, and then we have the student’s solution. The student’s solution is incorrect because he has calculated the maintenance cost to be 100,000 plus 100x, but actually, it should be 10x, because it’s only $10 per square foot, where x is the kind of size of the insulation in square feet, as they’ve defined it. This should actually be 360x plus 100,000, not 450x. If we run this code, the model says the student’s solution is correct.

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = model.generate_content(prompt)
print(response.text)


The student's solution is **almost correct**, but there's a small error in the maintenance cost calculation.

Here's the breakdown:

* **Land cost:** 100x (correct)
* **Solar panel cost:** 250x (correct)
* **Maintenance cost:** 100,000 + **10x** (**not** 100x). The student incorrectly used 100 instead of 10 for the per-square-foot maintenance cost.

**Corrected Total cost:** 100x + 250x + 100,000 + 10x = **360x + 100,000** 

**Therefore, the correct total cost for the first year of operations as a function of the number of square feet is 360x + 100,000.** 



The model agreed with the student's answer because it just kind of skim-read it. We can fix this by instructing the model to work out its solution first, and then compare its solution to the student’s solution.

We can do it by the prompt below. This prompt is a lot longer. In this prompt, we inform the model that your task is to determine if the student’s solution is correct or not. First, work out your own solution to the problem. Then, compare your solution to the student’s solution and evaluate if the student’s solution is correct or not.

Don’t decide if the student’s solution is correct until you have done the problem yourself. We have used the same trick to use the following format. The format will be the question, the student’s solution, the actual solution, and then whether the solution agrees, yes or no, and then the student's grade, correct or incorrect. Let's run the following prompt and see the answer by the model.

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = model.generate_content(prompt)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "finish_reason": 4,
          "index": 0,
          "safety_ratings": [],
          "token_count": 0,
          "grounding_attributions": []
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 399,
        "total_token_count": 399,
        "candidates_token_count": 0
      }
    }),
)
